<a href="https://colab.research.google.com/github/kartikbatra056/AllAboutTimeSeries/blob/main/MultiStepTimeSeriesWithLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d 'rohanrao/nifty50-stock-market-data'

 50% 9.00M/18.0M [00:00<00:00, 31.5MB/s]
100% 18.0M/18.0M [00:00<00:00, 51.9MB/s]


In [5]:
! unzip '/content/nifty50-stock-market-data.zip'

Archive:  /content/nifty50-stock-market-data.zip
  inflating: ADANIPORTS.csv          
  inflating: ASIANPAINT.csv          
  inflating: AXISBANK.csv            
  inflating: BAJAJ-AUTO.csv          
  inflating: BAJAJFINSV.csv          
  inflating: BAJFINANCE.csv          
  inflating: BHARTIARTL.csv          
  inflating: BPCL.csv                
  inflating: BRITANNIA.csv           
  inflating: CIPLA.csv               
  inflating: COALINDIA.csv           
  inflating: DRREDDY.csv             
  inflating: EICHERMOT.csv           
  inflating: GAIL.csv                
  inflating: GRASIM.csv              
  inflating: HCLTECH.csv             
  inflating: HDFC.csv                
  inflating: HDFCBANK.csv            
  inflating: HEROMOTOCO.csv          
  inflating: HINDALCO.csv            
  inflating: HINDUNILVR.csv          
  inflating: ICICIBANK.csv           
  inflating: INDUSINDBK.csv          
  inflating: INFRATEL.csv            
  inflating: INFY.csv                
 

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
plt.style.use('ggplot')

In [101]:
df=pd.read_csv('/content/RELIANCE.csv',parse_dates=['Date'])
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2000-01-03,RELIANCE,EQ,233.05,237.50,251.70,237.50,251.70,251.70,249.37,4456424,1.111319e+14,NaN,NaN,NaN
1,2000-01-04,RELIANCE,EQ,251.70,258.40,271.85,251.30,271.85,271.85,263.52,9487878,2.500222e+14,NaN,NaN,NaN
2,2000-01-05,RELIANCE,EQ,271.85,256.65,287.90,256.65,286.75,282.50,274.79,26833684,7.373697e+14,NaN,NaN,NaN
3,2000-01-06,RELIANCE,EQ,282.50,289.00,300.70,289.00,293.50,294.35,295.45,15682286,4.633254e+14,NaN,NaN,NaN
4,2000-01-07,RELIANCE,EQ,294.35,295.00,317.90,293.00,314.50,314.55,308.91,19870977,6.138388e+14,NaN,NaN,NaN


In [102]:
scaler=MinMaxScaler()
def PreprocessTimeSeries(data,split_ratio=0.2):

      split_size=int(split_ratio*data.shape[0])
      
      train=pd.DataFrame(index=data.iloc[:-split_size].index)
      valid=pd.DataFrame(index=data.iloc[-split_size:].index)
      
      train['VWAP']=scaler.fit_transform(data.iloc[:-split_size].values.reshape((train.shape[0],1)))
      valid['VWAP']=scaler.transform(data.iloc[-split_size:].values.reshape((valid.shape[0],1)))

      return train,valid

In [103]:
class MultistepDataset(Dataset):
      def __init__(self,data,T=10,n_out=5,step=1):
              
              super().__init__()

              self.data=data.values.reshape((data.shape[0],1))
              self.chunk_set=torch.FloatTensor(self.data).unfold(0,T+n_out,step).permute(0,2,1)
              
              self.chunks=(self.chunk_set[:,:T,:],self.chunk_set[:,T:T+n_out,:])
      
      def __getitem__(self,index:int):
              
              x=self.chunks[0][index]
              y=self.chunks[1][index]

              return x,y

      def __len__(self):

            return len(self.chunk_set)        

In [104]:
train,valid=PreprocessTimeSeries(df['VWAP'])
train.shape,valid.shape

((4148, 1), (1036, 1))

In [105]:
train_data=MultistepDataset(train)
valid_data=MultistepDataset(valid)

In [106]:
train_loader=DataLoader(train_data,batch_size=64)
valid_loader=DataLoader(valid_data,batch_size=64)

In [107]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [108]:
class EncoderModel(nn.Module):
      
      def __init__(self,n_stocks,hidden_dim,n_layers=1,drop_l=0.0):
           
           super(EncoderModel,self).__init__()
           self.n_layers=n_layers
           self.hidden_dim=hidden_dim
           self.encoder=nn.LSTM(n_stocks,hidden_size=hidden_dim,num_layers=n_layers,dropout=drop_l,batch_first=True)
           self.hidden=None

      def forward(self,Input):
      
           self.hidden=self.init_weights(Input.shape[0]) 
           _,self.hidden=self.encoder(Input,self.hidden)

           return self.hidden     

      def init_weights(self,batch_size):    

              return (torch.zeros((self.n_layers,batch_size,self.hidden_dim)).to(device),
                      torch.zeros((self.n_layers,batch_size,self.hidden_dim)).to(device))

In [109]:
class DecoderModel(nn.Module):
      
       def __init__(self,n_stocks,hidden_dim,n_layers=1,drop_l=0.0):
              
              super(DecoderModel,self).__init__()
              self.n_layers=n_layers
              self.hidden_dim=hidden_dim
              self.decoder=nn.LSTM(n_stocks,hidden_size=hidden_dim,num_layers=n_layers,dropout=drop_l,batch_first=True)
              self.fc=nn.Linear(hidden_dim,n_stocks)

       def forward(self,Input,hidden):

              output,hidden=self.decoder(Input,hidden) 
              output=self.fc(output)

              return output,hidden

In [138]:
n_stocks=1
hidden_dim=64
n_layers=1
drop_l=0.0
encoder=EncoderModel(n_stocks,hidden_dim,n_layers,drop_l)
decoder=DecoderModel(n_stocks,hidden_dim,n_layers,drop_l)
encoder=encoder.to(device)
decoder=decoder.to(device)
criterion=nn.MSELoss()
en_optim=optim.SGD(encoder.parameters(),lr=0.3)
de_optim=optim.SGD(decoder.parameters(),lr=0.003)
en_scheduler=optim.lr_scheduler.ReduceLROnPlateau(en_optim,mode='min')

In [139]:
def valid_model():
     encoder.eval()
     decoder.eval()
     with torch.no_grad():
         val_loss=0.0
         epoch_loss=0.0
         for x,y in valid_loader:
              
              x=x.to(device)
              y=y.to(device)
              
              hidden=encoder(x)
              output=x[:,-1,:].unsqueeze(1)

              loss=0.0

              for j in range(y.size(1)):
                  output,hidden = decoder(output,hidden)
                  loss+=criterion(y[:,j,:],output.view(-1,1))
              
              epoch_loss+=(loss.item()/y.size(1))*y.size(0)

         val_loss=epoch_loss/len(valid_data)

         return val_loss

In [140]:
def train_model(num_epochs):
     
     for i in range(num_epochs):
         encoder.train()
         decoder.train()
         train_loss=0.0
         epoch_loss=0.0
         val_loss=0.0
         for x,y in train_loader:
              
              x=x.to(device)
              y=y.to(device)
              
              en_optim.zero_grad()
              de_optim.zero_grad()

              hidden=encoder(x)
              output=x[:,-1,:].unsqueeze(1)

              loss=0.0

              for j in range(y.size(1)):
                  output,hidden = decoder(output,hidden)
                  loss+=criterion(y[:,j,:],output.view(-1,1))
              
              loss.backward()

              en_optim.step()
              de_optim.step()

              epoch_loss+=(loss.item()/y.size(1))*y.size(0)

         train_loss=epoch_loss/len(train_data)

         val_loss=valid_model()   
         
         en_scheduler.step(val_loss)

         print(f'Epoch No is {i+1}: \n \tTrain Loss: {train_loss} \n \tValid Loss: {val_loss}')

In [141]:
num_epochs=10
if __name__=='__main__':
    train_model(num_epochs)

Epoch No is 1: 
 	Train Loss: 0.033166072458338795 
 	Valid Loss: 0.02840493326084488
Epoch No is 2: 
 	Train Loss: 0.02682607865738707 
 	Valid Loss: 0.023063032813151522
Epoch No is 3: 
 	Train Loss: 0.024130409843201396 
 	Valid Loss: 0.02063766330072325
Epoch No is 4: 
 	Train Loss: 0.0215585884686116 
 	Valid Loss: 0.01868067568864258
Epoch No is 5: 
 	Train Loss: 0.018564455934953844 
 	Valid Loss: 0.016366770230743986
Epoch No is 6: 
 	Train Loss: 0.015372419658965866 
 	Valid Loss: 0.013591536404218693
Epoch No is 7: 
 	Train Loss: 0.012532127556924082 
 	Valid Loss: 0.010818601941321695
Epoch No is 8: 
 	Train Loss: 0.010338382276123584 
 	Valid Loss: 0.008394956968564342
Epoch No is 9: 
 	Train Loss: 0.00869130699191987 
 	Valid Loss: 0.006406002431203941
Epoch No is 10: 
 	Train Loss: 0.007393612819113262 
 	Valid Loss: 0.004887290153863845
